In [1]:
import pandas as pd
import numpy as np

In [168]:
# load data set
data = pd.read_csv(r"D:\Data\uber_rides_data.xlsx - sample_train.csv")
data.head()

,ride_id,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


In [3]:
# What is the shape of given dataset?
data.shape

(200000, 8)

In [4]:
# How many integer columns(by default) are given in the dataset?
data.info()
#ride_id,passenger_count are integer columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ride_id            200000 non-null  int64  
 1   fare_amount        200000 non-null  float64
 2   pickup_datetime    200000 non-null  object 
 3   pickup_longitude   200000 non-null  float64
 4   pickup_latitude    200000 non-null  float64
 5   dropoff_longitude  199999 non-null  float64
 6   dropoff_latitude   199999 non-null  float64
 7   passenger_count    200000 non-null  int64  
dtypes: float64(5), int64(2), object(1)
memory usage: 12.2+ MB


In [5]:
# How many missing values exists in 'dropoff_longitude' column?
data.isnull().sum()

ride_id              0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    1
dropoff_latitude     1
passenger_count      0
dtype: int64

In [134]:
# What is the data type of ' pickup_datetime' feature in your data?
data.pickup_datetime.dtype 
# object

dtype('O')

In [169]:
# Which of the following is the correct syntax to convert 'pickup_datetime' to datetime datatype?
data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'])

In [170]:
data.pickup_datetime.dtype 

datetime64[ns, UTC]

In [21]:
data['pickup_datetime'].astype('datetime64[ns]')

C:\Users\ASUS\AppData\Local\Temp\ipykernel_1856\3756586291.py:1: FutureWarning: Using .astype to convert from timezone-aware dtype to timezone-naive dtype is deprecated and will raise in a future version.  Use obj.tz_localize(None) or obj.tz_convert('UTC').tz_localize(None) instead
  data['pickup_datetime'].astype('datetime64[ns]')


0        2015-05-07 19:52:06
1        2009-07-17 20:04:56
2        2009-08-24 21:45:00
3        2009-06-26 08:22:21
4        2014-08-28 17:47:00
                 ...        
199995   2012-10-28 10:49:00
199996   2014-03-14 01:09:00
199997   2009-06-29 00:42:00
199998   2015-05-20 14:56:25
199999   2010-05-15 04:08:00
Name: pickup_datetime, Length: 200000, dtype: datetime64[ns]

In [171]:
# Which function can be used to remove null values from the dataframe?
data=data.dropna()

In [172]:
data.isnull().sum()

ride_id              0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64

In [164]:
data.shape

(199999, 13)

In [27]:
# What is the average fare amount?
#Remove the null values from the dataframe to answer the following question
data.fare_amount.sum()/data.fare_amount.count()

11.359891549457748

```
Calculate distance between each pickup and dropoff points using Haversine formula. 
What is the median haversine distance between pickup and dropoff location according to the given dataset?
```
Read about Haversine Distance here: https://en.wikipedia.org/wiki/Haversine_formula

In [173]:
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    d_lon = lon2 - lon1
    d_lat = lat2 - lat1
    a = np.sin(d_lat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(d_lon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    # Radius of Earth in kilometers
    r = 6371.0
    
    distance = r * c
    return distance

In [44]:
data.columns

Index(['ride_id', 'fare_amount', 'pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count'],
      dtype='object')

In [174]:
haversine_distances = haversine(data.pickup_latitude,data.pickup_longitude,data.dropoff_latitude,data.dropoff_longitude)

In [175]:
data['haversine_distances'] = haversine_distances

In [47]:
np.median(haversine_distances)

2.1209923961833708

In [50]:
# What is the maximum haversine distance between pickup and dropoff location according to the given dataset?
round(np.max(haversine_distances),2)

16409.24

In [56]:
# How many rides have 0.0 haversine distance between pickup and dropoff location according to the given dataset?
(haversine_distances == 0.0).sum()

5632

In [155]:
# What is the mean 'fare_amount' for rides with 0 haversine distance?
# Do you sense something fishy? Try to analyze, and give your expert opinion in Jupyter Notebook.


zero_distance_rides = data[data['haversine_distances'] == 0.0]

round(zero_distance_rides['fare_amount'].mean(),2)


11.59

In [62]:
# What is the maximum 'fare_amount' for a ride?
data['fare_amount'].max()

499.0

In [91]:
# What is the haversine distance between pickup and dropoff location for the costliest ride?
#Do you sense something fishy? Try to analyze, and give your expert opinion in Jupyter Notebook.

costliest_ride = data.sort_values(by='fare_amount', ascending=False).iloc[0]

# Retrieve the coordinates
pickup_lat = costliest_ride['pickup_latitude']
pickup_lon = costliest_ride['pickup_longitude']
dropoff_lat = costliest_ride['dropoff_latitude']
dropoff_lon = costliest_ride['dropoff_longitude']

# Calculate the Haversine distance
haversine(pickup_lat, pickup_lon, dropoff_lat, dropoff_lon)

0.0007899213191009994

In [71]:
# How many rides were recorded in the year 2014?
data['pickup_year'] = data['pickup_datetime'].dt.year
len(data[data['pickup_year'] == 2014])

29968

In [73]:
# How many rides were recorded in the first quarter of 2014?

data['pickup_year'] = data['pickup_datetime'].dt.year
data['pickup_quarter'] = data['pickup_datetime'].dt.quarter

len(data[(data['pickup_year'] == 2014) & (data['pickup_quarter'] == 1)])

7687

In [136]:
# On which day of the week in September 2010, maximum rides were recorded 
data['day_of_week'] = data['pickup_datetime'].dt.day_name()
data['month'] = data['pickup_datetime'].dt.month
data['year'] = data['pickup_datetime'].dt.year
september = data[(data['month'] == 9) & (data['year'] == 2010)]
september.groupby('day_of_week')['day_of_week'].count()

day_of_week
Friday       354
Monday       265
Saturday     362
Sunday       331
Thursday     457
Tuesday      322
Wednesday    391
Name: day_of_week, dtype: int64

In [176]:
data['ride_week_day'] = data['pickup_datetime'].dt.dayofweek
data['ride_week_day'] 

0         3
1         4
2         0
3         4
4         3
         ..
199995    6
199996    4
199997    0
199998    2
199999    5
Name: ride_week_day, Length: 199999, dtype: int64

```
Apply a Machine Learning Algorithm to predict the fare amount given following input features:
passenger_count, distance and ride_week_day.

Perform a 70-30 split of data.

Which algorithm gives the least adjusted R square value?
```

In [177]:
data.columns

Index(['ride_id', 'fare_amount', 'pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count', 'haversine_distances', 'ride_week_day'],
      dtype='object')

In [180]:
# Identifying the inputs (X) and output (y)
X = data[['passenger_count', 'haversine_distances', 'ride_week_day']]
y = data['fare_amount']

In [187]:
# split into train and test

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.70, random_state=100)

In [188]:
X_train.dtypes

passenger_count          int64
haversine_distances    float64
ride_week_day            int64
dtype: object

In [191]:
# scaling the numerical features
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# column names are (annoyingly) lost after Scaling
# (i.e. the dataframe is converted to a numpy ndarray)

X_train_transformed = pd.DataFrame(scaler.fit_transform(X_train), 
                                    columns = X_train.columns, 
                                    index = X_train.index)
X_train_transformed.head()

,passenger_count,haversine_distances,ride_week_day
195896,1.008439,-0.049696,1.001584
47114,0.242972,-0.044699,-1.053422
144876,0.242972,-0.051539,-0.025919
184791,0.242972,-0.052902,1.515335
123270,-0.522495,-0.052840,-1.053422


In [196]:
X_test_transformed = pd.DataFrame(scaler.transform(X_test), 
                                   columns = X_test.columns, 
                                   index = X_test.index)
X_test_transformed.head()

,passenger_count,haversine_distances,ride_week_day
180481,-0.522495,-0.052349,1.515335
8971,-0.522495,-0.048410,1.515335
77027,2.539373,-0.045392,1.001584
21656,1.008439,-0.043991,-0.025919
58303,-0.522495,-0.046622,-1.053422


# Decision Tree Regression

In [215]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor()
regressor.fit(X_train_transformed, y_train)
y_train_pred = regressor.predict(X_train_transformed)
y_test_pred = regressor.predict(X_test_transformed)

In [216]:
# Adjusted R^2
def Adjusted_r2(y_train,y_train_pred,X_train_transform):
    from sklearn import metrics
    r2 = metrics.r2_score(y_train, y_train_pred)
    n = len(y_train)
    k = X_train_transform.shape[1]

    r2_adj = 1 - (1-r2)*(n-1)/(n-k-1)
    return r2_adj

In [217]:
DT_tr_score = Adjusted_r2(y_train,y_train_pred,X_train_transformed)
print("Train Score:",DT_tr_score*100)

DT_te_score = Adjusted_r2(y_test,y_test_pred,X_test_transformed)
print("Test Score:",DT_te_score*100)

Train Score: 93.93946090952012
Test Score: 50.62908910319134


# Random Forest Regression

In [218]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor()
regressor.fit(X_train_transformed, y_train)
y_train_pred = regressor.predict(X_train_transformed)
y_test_pred = regressor.predict(X_test_transformed)

RF_tr_score = Adjusted_r2(y_train,y_train_pred,X_train_transformed)
print("Train Score:",RF_tr_score*100)

RF_te_score = Adjusted_r2(y_test,y_test_pred,X_test_transformed)
print("Test Score:",RF_te_score*100)

Train Score: 90.59216910741964
Test Score: 64.35041459365883


In [219]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

tuned_parameters = [{'max_depth' : [1,2,3,4,5,6,7,8,9]}]

regressor_RF = GridSearchCV(
    estimator=RandomForestRegressor(),
    param_grid=tuned_parameters, 
    scoring='r2',
    cv=5,
    return_train_score=True,
    verbose=1
)
regressor_RF.fit(X_train_transformed, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid=[{'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9]}],
             return_train_score=True, scoring='r2', verbose=1)

In [220]:
print("Best parameters set found on train set")
print(regressor_RF.best_params_)
print(regressor_RF.best_estimator_)
print()
RF_Test_Score =  regressor_RF.score(X_test_transformed, y_test)*100
print('Score on Test Data: ',RF_Test_Score)

Best parameters set found on train set
{'max_depth': 7}
RandomForestRegressor(max_depth=7)

Score on Test Data:  69.68068676525618


# Linear regression

In [223]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train_transformed, y_train)
y_train_pred = regressor.predict(X_train_transformed)
y_test_pred = regressor.predict(X_test_transformed)


LR_tr_score = Adjusted_r2(y_train,y_train_pred,X_train_transformed)
print("Train Score:",LR_tr_score*100)

LR_te_score = Adjusted_r2(y_test,y_test_pred,X_test_transformed)
print("Test Score:",LR_te_score*100)

Train Score: 0.08022069160746881
Test Score: 0.05416156910665881


# Conclusion
**Random Forest Regression** algorithm is the best model.